In [1]:
!pip install qiskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 15.1 MB/s eta 0:00:00


In [2]:
#superpositon, entanglement and interference
my_list=[1,3,5,2,4,9,5,8,0,7,6]

**Classical Computing**\
Naive approach\
Requires N/2 average number of times oracle to be called\
but in worst case N times



In [3]:
#eracle is the black box at the disposal->we can call it and ask whether this number is a whinner or not
def the_oracle(my_input):
  winner=7
  if my_input is winner:
    response=True
  else:
    response=False
  return response

In [4]:
for index, trial_number in enumerate(my_list):
  if the_oracle(trial_number)==True:
    print('winner found at index %i'%index)
    print('%i calls at the oracle used'%(index+1))

winner found at index 9
10 calls at the oracle used


**Quantum Computing**\
O(SQRT(N))

In [5]:
#oracle works here by flipping the sign of the input of the winner
#controlled Z gate -> |11> -> [CZ] -> -|11>
#amplitute amplification ->reflection operator

#Grovvers diffusion operator-> oracle+ Reflection
from qiskit import *
import matplotlib.pyplot as py
import numpy as np

In [7]:
#define oracle circuit
oracle = QuantumCircuit(2,name='oracle')
oracle.cz(0,1)
oracle.to_gate()#make oracle into its own gate

Instruction(name='oracle', num_qubits=2, num_clbits=0, params=[])

In [9]:
oracle.draw(initial_state = True)

q_0: |0>─■─
         │ 
q_1: |0>─■─

In [10]:
#preparing a supepositon state of all the gates by applying the handmard gate on each one of them
#in this way each uery xould be simuntaneousky reached oracle
backend=Aer.get_backend('statevector_simulator')
# statevector_simulator backend is selected, which allows for simulation of the quantum state vector of the circuit.
grover_circ=QuantumCircuit(2,2)
#The first argument 2 specifies the number of qubits in the quantum registers, and the second argument 2 specifies the number of classical bits in the classical registers.
grover_circ.h([0,1])
#This line applies the Hadamard gate (H gate) to qubits 0 and 1 in the grover_circ circuit.
grover_circ.append(oracle,[0,1])
grover_circ.draw(initial_state = True)

┌───┐┌─────────┐
q_0: |0>┤ H ├┤0        ├
        ├───┤│  oracle │
q_1: |0>┤ H ├┤1        ├
        └───┘└─────────┘
 c: 0 2/════════════════

In [11]:
job= execute(grover_circ,backend)
result=job.result()

In [12]:
sv=result.get_statevector()
np.around(sv,2)

array([ 0.5+0.j,  0.5+0.j,  0.5+0.j, -0.5+0.j])

In [16]:
#square the state vectors to get back the probabilities
reflection = QuantumCircuit(2,name="reflection")
reflection.h([0,1])
reflection.z([0,1])
reflection.cz(0,1)
reflection.h([0,1])
reflection.to_gate()

Instruction(name='reflection', num_qubits=2, num_clbits=0, params=[])

In [17]:
reflection.draw(initial_state = True)

┌───┐┌───┐   ┌───┐
q_0: |0>┤ H ├┤ Z ├─■─┤ H ├
        ├───┤├───┤ │ ├───┤
q_1: |0>┤ H ├┤ Z ├─■─┤ H ├
        └───┘└───┘   └───┘

In [18]:
backend=Aer.get_backend('statevector_simulator')
# statevector_simulator backend is selected, which allows for simulation of the quantum state vector of the circuit.
grover_circ=QuantumCircuit(2,2)
#The first argument 2 specifies the number of qubits in the quantum registers, and the second argument 2 specifies the number of classical bits in the classical registers.
grover_circ.h([0,1])
#This line applies the Hadamard gate (H gate) to qubits 0 and 1 in the grover_circ circuit.
grover_circ.append(oracle,[0,1])
grover_circ.append(reflection,[0,1])
grover_circ.measure([0,1],[0,1])

In [19]:
grover_circ.draw(initial_state = True)

┌───┐┌─────────┐┌─────────────┐┌─┐   
q_0: |0>┤ H ├┤0        ├┤0            ├┤M├───
        ├───┤│  oracle ││  reflection │└╥┘┌─┐
q_1: |0>┤ H ├┤1        ├┤1            ├─╫─┤M├
        └───┘└─────────┘└─────────────┘ ║ └╥┘
 c: 0 2/════════════════════════════════╩══╩═
                                        0  1

In [20]:
job= execute(grover_circ,backend,shots=1)
result=job.result()
result.get_counts()

{'11': 1}